In [20]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
import os
import numpy as np

In [21]:
class HandGestureDataset(Dataset):
    def __init__(self,filepath):
        xy=np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len=xy.shape[0]
        self.x_data=torch.from_numpy(xy[:,:-1])
        self.y_data=torch.from_numpy(xy[:,[-1]])

    def __getitem__(self, index):
        return self.x_data[index],self.y_data[index]

    def __len__(self):
        return self.len

Training_set=HandGestureDataset('data\Training_set\Training_set.csv')
train_loader=DataLoader(dataset=Training_set,shuffle=1,num_workers=0)

Test_set=HandGestureDataset('data\Test_set\Test_set.csv')
test_loader=DataLoader(dataset=Test_set,batch_size=22,shuffle=1,num_workers=0)

In [22]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1=torch.nn.Linear(63,40)
        self.l2=torch.nn.Linear(40,20)
        self.l3=torch.nn.Linear(20,10)
        self.l4=torch.nn.Linear(10,1)
        self.sigmoid=torch.nn.Sigmoid()

    def forward(self,x):
        x=self.sigmoid(self.l1(x))
        x=self.sigmoid(self.l2(x))
        x=self.sigmoid(self.l3(x))
        return self.sigmoid(self.l4(x))

model=Net()

In [23]:
criterion=torch.nn.BCELoss()
optimizer=optim.SGD(model.parameters(),lr=0.005)

In [24]:
def train(epoch):
    running_loss=0.0
    for batch_idx, data in enumerate(train_loader,0):
        inputs,target=data

        optimizer.zero_grad()

        outputs=model(inputs)
        loss=criterion(outputs,target)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

        if batch_idx % 40==39:
            print('[%d, %5d] loss: %.3f'%(epoch+1,batch_idx+1,running_loss))
            running_loss=0

def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            inputs,labels=data
            labels=labels.long()
            
            outputs=model(inputs)
            predicted=torch.Tensor(22,1)
            predicted=outputs.data>0.5
            total+=labels.size(0)
            correct+=(predicted.long()==labels).sum().item()
    
    print('Accuracy on test set: %d %%'%(100*correct/total))
    

In [25]:
if __name__=='__main__':
    for epoch in range(500):
        train(epoch)
        test()
        

[1,    40] loss: 27.816
[1,    80] loss: 27.807
[1,   120] loss: 27.793
Accuracy on test set: 50 %
[2,    40] loss: 27.816
[2,    80] loss: 27.780
[2,   120] loss: 27.844
Accuracy on test set: 50 %
[3,    40] loss: 27.793
[3,    80] loss: 27.835
[3,   120] loss: 27.801
Accuracy on test set: 45 %
[4,    40] loss: 27.800
[4,    80] loss: 27.748
[4,   120] loss: 27.880
Accuracy on test set: 50 %
[5,    40] loss: 27.585
[5,    80] loss: 27.911
[5,   120] loss: 27.862
Accuracy on test set: 50 %
[6,    40] loss: 27.821
[6,    80] loss: 27.726
[6,   120] loss: 27.905
Accuracy on test set: 50 %
[7,    40] loss: 27.815
[7,    80] loss: 27.760
[7,   120] loss: 27.862
Accuracy on test set: 50 %
[8,    40] loss: 27.791
[8,    80] loss: 27.784
[8,   120] loss: 27.861
Accuracy on test set: 50 %
[9,    40] loss: 27.654
[9,    80] loss: 27.891
[9,   120] loss: 27.848
Accuracy on test set: 50 %
[10,    40] loss: 27.477
[10,    80] loss: 28.090
[10,   120] loss: 27.811
Accuracy on test set: 50 %
[11,   

In [27]:
torch.save(model,f='model.pkl') 